In [1]:
pip install --upgrade pip


     ---------------------------------------- 2.1/2.1 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric


Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
  Using cached torch_sparse-0.6.17.tar.gz (209 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for torch-sparse
Failed to build torch-sparse


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [55 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.9
      creating build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\add.py -> build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\bandwidth.py -> build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\cat.py -> build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\coalesce.py -> build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\convert.py -> build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\diag.py -> build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\eye.py -> build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\index_select.py -> build\lib.win-amd64-3.9\torch_sparse
      copying torch_sparse\masked_select.py -> build\lib.wi

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
  Using cached torch_scatter-2.1.1.tar.gz (107 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for torch-scatter
Failed to build torch-scatter


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [32 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.9
      creating build\lib.win-amd64-3.9\torch_scatter
      copying torch_scatter\placeholder.py -> build\lib.win-amd64-3.9\torch_scatter
      copying torch_scatter\scatter.py -> build\lib.win-amd64-3.9\torch_scatter
      copying torch_scatter\segment_coo.py -> build\lib.win-amd64-3.9\torch_scatter
      copying torch_scatter\segment_csr.py -> build\lib.win-amd64-3.9\torch_scatter
      copying torch_scatter\testing.py -> build\lib.win-amd64-3.9\torch_scatter
      copying torch_scatter\utils.py -> build\lib.win-amd64-3.9\torch_scatter
      copying torch_scatter\__init__.py -> build\lib.win-amd64-3.9\torch_scatter
      creating build\lib.win-amd64-3.9\torch_scatter\composite
      copying torch_scatter\composite\log

In [4]:
%matplotlib inline
import torch
import networkx as nx
import matplotlib.pyplot as plt